In [51]:
import pandas as pd
import json

with open('naukri_apply_links_20260201_233500.json', 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data)
df.head()

,job_id,title,company,job_url,apply_type,apply_link,error
0,040225514997,AI Engineer,Accely,https://www.naukri.com/job-listings-ai-enginee...,external_popup,chrome-error://chromewebdata/,NaN
1,300126503289,AI Engineer,Opplane,https://www.naukri.com/job-listings-ai-enginee...,external_popup,https://opplane.factorialhr.pt/job_posting/ai-...,NaN
2,171024500497,AI Engineer,Kodo,https://www.naukri.com/job-listings-ai-enginee...,no_apply_button,None,NaN
3,080825503562,AI Engineer,No Limit Technology,https://www.naukri.com/job-listings-ai-enginee...,external_popup,https://nolimitstaffing.zohorecruit.com/recrui...,NaN
4,270825501819,AI Engineer,Sysfort Systems,https://www.naukri.com/job-listings-ai-enginee...,external_popup,chrome-error://chromewebdata/,NaN


In [52]:
df_expanded = pd.json_normalize(df['job_details'])
df_expanded.head()

KeyError: 'job_details'

In [37]:
df2 = pd.concat([df, df_expanded], axis=1)

In [38]:
df2.drop(columns=['job_details','job_id'], inplace=True)

In [53]:
df.to_csv('test4.csv',index=False)

### Extract Email ID

In [40]:
df = pd.read_csv('test.csv')
df.head()

,apply_link,title,company,emails,description,skills
0,NaN,AI and Machine Learning Engineering Professional,Influencermarketing.Ai,https://www.google.com/search?q=Influencermark...,Overview\n\n\n\nInfluencer Marketing AI is bui...,['Key Skills\nComputer visiondata scienceMachi...
1,https://www.naukri.com/myapply/showAcp?jquery=...,Freelance Artificial Intelligence,Digital Solution Tech,https://www.google.com/search?q=Digital Soluti...,Job Title: Freelance Artificial Intelligence S...,['Key Skills\nComputer scienceComputer visionD...
2,https://www.naukri.com/myapply/showAcp?jquery=...,Freelance Agent Evaluation Engineer,Mindrift,https://www.google.com/search?q=Mindrift+caree...,we're looking for someone who can design reali...,['Key Skills\nData analysisAnalyticalMachine l...
3,https://www.naukri.com/myapply/showAcp?jquery=...,Data Scientist,Sanfoundry,https://www.google.com/search?q=Sanfoundry+car...,Job description:\nPreferred Qualifications:\nK...,['Key Skills\nSupply chainData analysisSales o...
4,https://www.naukri.com/myapply/showAcp?jquery=...,AI Engineer (Voice Expertise),Zone It,https://www.google.com/search?q=Zone It+career...,About the Role\n\nWe are looking for an experi...,['Key Skills\nComputer scienceERPProduct engin...


In [42]:
# !pip install pandas playwright beautifulsoup4 tldextract
# !playwright install


In [46]:
import asyncio
import pandas as pd
import re
from playwright.async_api import async_playwright

EMAIL_REGEX = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"

async def get_company_emails(company_name):
    search_url = f"https://www.google.com/search?q={company_name}+careers+email"

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(search_url, timeout=60000)

        content = await page.content()
        await browser.close()

    emails = set(re.findall(EMAIL_REGEX, content))

    career_emails = [
        e for e in emails
        if any(k in e.lower() for k in ["career", "hr", "jobs", "talent"])
    ]

    return ", ".join(career_emails) if career_emails else ""


In [47]:
async def process_csv():
    df = pd.read_csv("test.csv")
    df['company'] = df['company'].str.replace(r'\n\d+\.?\d*\s+Reviews?', '', regex=True).str.strip()
    df["career_email"] = ""

    for idx, row in df.iterrows():
        company = row["company"]
        print(f"Searching: {company}")
        email = await get_company_emails(company)
        print(f"Found: {email}")
        df.at[idx, "career_email"] = email

        # Save after every row (no data loss)
        df.to_csv("output_with_emails.csv", index=False)

asyncio.run(process_csv())


RuntimeError: asyncio.run() cannot be called from a running event loop

In [48]:
# Clean the company column by removing ratings and reviews text
df['company'] = df['company'].str.replace(r'\n\d+\.?\d*\s+Reviews?', '', regex=True).str.strip()

# Display the cleaned company column
df['company'].head(20)

0                                Influencermarketing.Ai
1                                 Digital Solution Tech
2                                              Mindrift
3                                            Sanfoundry
4                                               Zone It
5                       Altayr Global Business Services
6              Navgurukul Foundation For Social Welfare
7                                 Digital Solution Tech
8                                            SNS System
9                                  Arivonix Corporation
10                                              Decoded
11                                            Speechify
12                                           Lion Group
13                                         Think Future
14    Allscripts(India) LLP, ultimately a subsidiary...
15                                Infraveo Technologies
16                             Robusta Technology Group
17                                              

### Resume Provide

In [54]:
df = pd.read_csv("output_with_emails.csv")
df1 = pd.read_csv("output_with_emails_2.csv")

df1.head()

,job_id,title,company,job_url,apply_type,apply_link,error,career_email
0,40225514997,AI Engineer,Accely,https://www.naukri.com/job-listings-ai-enginee...,external_popup,chrome-error://chromewebdata/,NaN,NaN
1,300126503289,AI Engineer,Opplane,https://www.naukri.com/job-listings-ai-enginee...,external_popup,https://opplane.factorialhr.pt/job_posting/ai-...,NaN,contact@opplane.com
2,171024500497,AI Engineer,Kodo,https://www.naukri.com/job-listings-ai-enginee...,no_apply_button,NaN,NaN,"jobs@kodo.in, jobs@kodo.com, careers@kodokids.com"
3,80825503562,AI Engineer,No Limit Technology,https://www.naukri.com/job-listings-ai-enginee...,external_popup,https://nolimitstaffing.zohorecruit.com/recrui...,NaN,NaN
4,270825501819,AI Engineer,Sysfort Systems,https://www.naukri.com/job-listings-ai-enginee...,external_popup,chrome-error://chromewebdata/,NaN,NaN


In [55]:
df.head()

,apply_link,title,company,emails,description,skills,career_email
0,NaN,AI and Machine Learning Engineering Professional,Influencermarketing.Ai,https://www.google.com/search?q=Influencermark...,Overview\n\n\n\nInfluencer Marketing AI is bui...,['Key Skills\nComputer visiondata scienceMachi...,NaN
1,https://www.naukri.com/myapply/showAcp?jquery=...,Freelance Artificial Intelligence,Digital Solution Tech,https://www.google.com/search?q=Digital Soluti...,Job Title: Freelance Artificial Intelligence S...,['Key Skills\nComputer scienceComputer visionD...,service.desk@curtin.edu.au
2,https://www.naukri.com/myapply/showAcp?jquery=...,Freelance Agent Evaluation Engineer,Mindrift,https://www.google.com/search?q=Mindrift+caree...,we're looking for someone who can design reali...,['Key Skills\nData analysisAnalyticalMachine l...,NaN
3,https://www.naukri.com/myapply/showAcp?jquery=...,Data Scientist,Sanfoundry,https://www.google.com/search?q=Sanfoundry+car...,Job description:\nPreferred Qualifications:\nK...,['Key Skills\nSupply chainData analysisSales o...,NaN
4,https://www.naukri.com/myapply/showAcp?jquery=...,AI Engineer (Voice Expertise),Zone It,https://www.google.com/search?q=Zone It+career...,About the Role\n\nWe are looking for an experi...,['Key Skills\nComputer scienceERPProduct engin...,careers@zoneitsolutions.com


In [64]:
df3 =pd.DataFrame({'company_email': df['company'] + ' - ' + df['career_email']})
df4 = pd.DataFrame({'company_email': df1['company'] + ' - ' + df1['career_email']})
df3.head()

,company_email
0,NaN
1,Digital Solution Tech - service.desk@curtin.ed...
2,NaN
3,NaN
4,Zone It - careers@zoneitsolutions.com


In [72]:
df3.dropna(inplace=True)
df4.dropna(inplace=True)
df5 = df3 + df4

In [74]:
# Split the company_email column into separate 'company' and 'email' columns
df5[['company', 'email']] = df5['company_email'].str.split(' - ', n=1, expand=True)

df5.head()

,company_email,company,email
1,Digital Solution Tech - service.desk@curtin.ed...,Digital Solution Tech,service.desk@curtin.edu.auOpplane - contact@op...
2,NaN,NaN,NaN
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN


In [71]:
df5.dropna(inplace=True)
df5.shape
df5['email'] = df5['email'].str.split(', ')
df5['email'].explode().reset_index(drop=True)
df5.head()

,company_email,company,email
1,Digital Solution Tech - service.desk@curtin.ed...,Digital Solution Tech,NaN
13,Think Future - recruiter@tftus.comPradisys Tec...,Think Future,NaN
17,"Decoded - jdoe@careersdecoded.com, jane.doe@ca...",Decoded,NaN
21,Matsci Ai - sgupta@matsci.aiLegalease Solution...,Matsci Ai,NaN
26,Rolling Arrays - india@rollingarrays.comFlytBa...,Rolling Arrays,NaN


In [61]:
df5.to_csv("combined_emails.csv", index=False)

### Automation

RuntimeError: asyncio.run() cannot be called from a running event loop

In [12]:
import pandas as pd
df = pd.read_csv('HR_List_urls.csv')
df.head()

,links,Unnamed: 1
0,https://opplane.factorialhr.pt/job_posting/ai-...,NaN
1,https://nolimitstaffing.zohorecruit.com/recrui...,NaN
2,https://www.databricks.com/company/careers/pro...,NaN
3,https://www.naukri.com/myapply/showAcp?jquery=...,NaN
4,https://jobs.micro1.ai/post/dea0a807-b12c-4b81...,NaN


In [ ]:
urls = df['links'].to_list()
urls

['https://opplane.factorialhr.pt/job_posting/ai-engineer-279754',
 'https://nolimitstaffing.zohorecruit.com/recruit/PortalDetail.na?iframe=true&digest=Aj98tVjKr5H2j0B4Yw55dnGAUnkE7bVo.a85bxdBEmQ-&widgetid=371351000000072311&jobid=474875000014941003',
 'https://www.databricks.com/company/careers/professional-services-operations/ai-engineer---fde-forward-deployed-engineer-8099751002?gh_jid=8099751002',
 'https://www.naukri.com/myapply/showAcp?jquery=1&file=270825501650&multiApplyResp={%22270825501650%22:202}',
 'https://jobs.micro1.ai/post/dea0a807-b12c-4b81-bc4c-ca8005f256fe',
 'https://onemagnify.wd5.myworkdayjobs.com/OneMagnify_Careers/job/AI-Engineer_R1995',
 'https://jobs.micro1.ai/post/dea0a807-b12c-4b81-bc4c-ca8005f256fe',
 'https://jobs.micro1.ai/post/b351c947-9e3c-4c40-a88a-4338a3c6d303',
 'https://pradisystechnologies.in/careers/HR-OPN-2026-0002',
 'https://jobs.micro1.ai/post/a4f5751c-c073-4f3e-9030-caf337fc4c44',
 'https://opplane.factorialhr.pt/job_posting/lead-ai-engineer-g